In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import time
import datetime

#    ----------------------  Градиентный бустинг   --------------------------------

data = pd.read_csv("C:/Users/asus/Desktop/coursera/week7/features.csv", index_col='match_id')
y = data['radiant_win']

data = data.drop(['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'], axis=1)
feature_sum = data.count()

feature_name = []
for i in data:
    feature_name.append(i)
    
feature_sum_array = []
for i in feature_sum:
    feature_sum_array.append(i)
    
gap = []
for i in range(len(feature_name)):
    if feature_sum_array[i] < len(data):
        gap.append(feature_name[i])

#список признаков имеющие пропуски
print (gap)

#замена пропусков на 0
X = data.fillna(value = 0)

#Генератор разбиений
kf = KFold(n_splits=5, shuffle=True)
kf.split(X)

tree_num = [10, 20, 30, 40, 50, 60]
for i in tree_num:
    clf = GradientBoostingClassifier(n_estimators=i)
    start_time = datetime.datetime.now()
    clf.fit(X, y)
    scores = cross_val_score(clf, X, y, scoring='roc_auc', cv=kf)
    print ('Time elapsed:', datetime.datetime.now() - start_time)
    print (np.mean(scores))
    
#    ----------------------  Логистическая регрессия   --------------------------------

#  ---- без мешка слов  ----

data = pd.read_csv("C:/Users/asus/Desktop/coursera/week7/features.csv", index_col='match_id')
y = data['radiant_win']

data = data.drop(['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'], axis=1)
a = np.unique(data['r1_hero'])
print(max(a))

#замена пропусков на 0
X = data.fillna(value = 0)
scaler = StandardScaler()
X = scaler.fit_transform (X)

kf = KFold(n_splits=5, shuffle=True)
kf.split(X)

start_time = datetime.datetime.now()
clf = LogisticRegression(penalty='l2', C = 1)
clf.fit(X, y)
scores = cross_val_score(clf, X, y, scoring='roc_auc', cv=kf)
print (np.mean(scores))
print ('Time elapsed:', datetime.datetime.now() - start_time)


#  ---- с мешком слов  ----
data = pd.read_csv("C:/Users/asus/Desktop/coursera/week7/features.csv", index_col='match_id')
data_test = pd.read_csv("C:/Users/asus/Desktop/coursera/week7/features_test.csv", index_col='match_id')
y = data['radiant_win']

h = np.unique(data['d1_hero'])
N = max(h)
X_pick = np.zeros((data.shape[0], N))
X_pick_test = np.zeros((data_test.shape[0], N))

for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
for i, match_id in enumerate(data_test.index):
    for p in range(5):
        X_pick_test[i, data_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, data_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

data = data.drop(['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'], axis=1)
data = data.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero' ],  axis=1)
data_test = data_test.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero' ],  axis=1)
#замена пропусков на 0
X = data.fillna(value = 0)
X_test = data_test.fillna(value = 0)
scaler = StandardScaler()
X = scaler.fit_transform (X)
X_test = scaler.fit_transform(X_test)

wordsBag = np.hstack((X,X_pick))
wordsBag_test = np.hstack((X_test,X_pick_test))

kf = KFold(n_splits=5, shuffle=True)
kf.split(X)

start_time = datetime.datetime.now()
clf = LogisticRegression(penalty='l2', C = 1)
clf.fit(wordsBag, y)
pred = clf.predict_proba(wordsBag_test)[:, 1]
scores = cross_val_score(clf, wordsBag, y, scoring='roc_auc', cv=kf)
print (np.mean(scores))
print ('Time elapsed:', datetime.datetime.now() - start_time)
print ('max', max(pred))
print ('min', min(pred))

['first_blood_time', 'first_blood_team', 'first_blood_player1', 'first_blood_player2', 'radiant_bottle_time', 'radiant_courier_time', 'radiant_flying_courier_time', 'radiant_first_ward_time', 'dire_bottle_time', 'dire_courier_time', 'dire_flying_courier_time', 'dire_first_ward_time']
Time elapsed: 0:00:37.578531
0.6644964601501167
Time elapsed: 0:01:09.561957
0.6816731893166407
Time elapsed: 0:01:39.710474
0.68989980774613
Time elapsed: 0:02:10.507075
0.6941649782019813
Time elapsed: 0:02:42.032831
0.6969957157452403
Time elapsed: 0:03:16.410927
0.7003225544019323
112
0.716204221991857
Time elapsed: 0:00:18.102608
0.7518100971889474
Time elapsed: 0:00:32.792334
max 0.9965116494292339
min 0.008614713620377102
